الفكرة ببساطة

في File Streaming، Spark بيعمل  على فولدر معين.
كل ما تحط ملف جديد في الفولدر ده، Spark يقرأه تلقائي ويعمل عليه الـ Processing اللي انت محددها.

💡 يعني بدل ما البيانات جاية من socket live، هي جاية من ملفات جديدة بتتحط في فولدر معين

In [1]:
# stream from files 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count

spark = SparkSession.builder \
    .appName("StreamingOrdersExample") \
    .getOrCreate()



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/31 20:26:16 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/10/31 20:26:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/31 20:26:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#الاول هنحدد الاسكميا بتاعتنا 
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer", StringType(), True),
    StructField("product", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", IntegerType(), True)
])

# قراءة الملفات كـ stream
orders = spark.readStream \
    .option("header", True) \
    .schema(schema) \
    .csv("/home/ahmed-refat/datastream")  # ← اسم الفولدر




# طباعة schema للتأكد
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer: string (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: integer (nullable = true)



In [9]:
# عرض البيانات على console كل 5 ثواني
query = (
    orders.writeStream
    .outputMode("complete")        # عرض الصفوف الجديدة فقط
    .format("console")           # عرض النتائج في الـ console
    .trigger(processingTime='10 seconds')  # تحديث كل 5 ثواني
    .start()
)


25/10/31 21:46:01 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4b8237d4-b4aa-4921-8c78-8cde85578323. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/31 21:46:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: [STREAMING_OUTPUT_MODE.UNSUPPORTED_OPERATION] Invalid streaming output mode: complete. This output mode is not supported for no streaming aggregations on streaming DataFrames/DataSets. SQLSTATE: 42KDE

In [7]:
query.stop()

25/10/31 21:44:32 WARN DAGScheduler: Failed to cancel job group 6163b4e3-2b4a-4ef2-b8db-3fb2a3bdc2c1. Cannot find active jobs for it.
25/10/31 21:44:33 WARN DAGScheduler: Failed to cancel job group 6163b4e3-2b4a-4ef2-b8db-3fb2a3bdc2c1. Cannot find active jobs for it.
